# Source :
https://github.com/williamscott701/Information-Retrieval/blob/master/2.%20TF-IDF%20Ranking%20-%20Cosine%20Similarity%2C%20Matching%20Score/TF-IDF.ipynb

https://github.com/mayank408/TFIDF/blob/master/TFIDF.ipynb

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
from sklearn.model_selection import *

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [5]:
# load the dataset
data = open('data/corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# create a dataframe using texts and lables
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels

# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = train_test_split(trainDF['text'], trainDF['label'])


In [6]:
trainDF

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2
...,...,...
9995,A revelation of life in small town America in ...,__label__2
9996,Great biography of a very interesting journali...,__label__2
9997,Interesting Subject; Poor Presentation: You'd ...,__label__1
9998,Don't buy: The box looked used and it is obvio...,__label__1


In [7]:

# docA = "The cat sat on my face"
# docB = "The dog sat on my bed"
# docC = "The cat sat on my face"
# docD = "The dog sat on my bed"

In [8]:
# data = pd.DataFrame()
# data['text'] = [docA,docB,docC,docD]
# data['label'] = ['a','b','a','b']
# data

In [9]:
isinstance(1,int)

True

In [21]:
def get_tftdfcf(data_frame):
    import math
    wordSet = {}
    word_count = []
    word_count_label = []
    tf_array = []
    idf_array = []
    tfidf_array = []

    for doc_2 in data_frame.text:
        doc_2 = doc_2.split(" ")
        wordSet = set(wordSet).union(set(doc_2))
    
    print(len(wordSet))
    
    for label in data_frame.label.unique():
        data_to_process = data_frame[data_frame.label == label]
        data_to_process.drop(columns = ['label'])
        wordDict_label = dict.fromkeys(wordSet, 0) 
        for doc_3 in data_to_process.text:
            doc_3 = doc_3.split(" ")
            for word in doc_3:
                if isinstance(wordDict_label[word],int):
                    wordDict_label[word]+=1
            wordDict_label['label'] = label
            wordDict_label['records'] = len(data_to_process)
        word_count_label.append(wordDict_label)
        # print(label , wordDict_label)
        
    for doc,label in zip(data_frame.text,data_frame.label):
        doc = doc.split(" ")
        wordDict = dict.fromkeys(wordSet, 0)
    
        for word in doc:
            wordDict[word]+=1
        wordDict['label'] = label
        word_count.append(wordDict)

        tfDict = {}
        bowCount = len(doc)
        for word, count in wordDict.items():
            if  word != 'label':
                tfDict[word] = math.log10(count/float(bowCount)+1.0)
            tfDict['label'] = label
        tf_array.append(tfDict)
        # print(wordDict)

    
    idfDict = {}
    N = len(data_frame)
    
    idfDict = dict.fromkeys(wordSet, 0)
    for doc in word_count:
        for word, val in doc.items():
            if word != 'label':
                if val > 0 :
                    idfDict[word] += 1

    for word, val in idfDict.items():
        if word  != 'label'  :
            idfDict[word] = math.log10(N / float(val)) if val > 0 else 0
    idf_array.append(idfDict)
    
    tf_df = pd.DataFrame(tf_array)
    cf_data = pd.DataFrame(word_count_label)

    for label_index in tf_df.label.unique():
        data_to_process = tf_df[tf_df['label'] == label_index]
        cf_data_to_process = cf_data[cf_data['label'] == label_index]
        cf_data_to_process = cf_data_to_process.drop(columns = ['label'])
        n_len = len(cf_data_to_process)
        if n_len == 1 :
            no_records_per_label = list(cf_data_to_process.records)[0]
            cf_data_to_process  = dict(cf_data_to_process.iloc[0,:])
        # print(no_records_per_label)
        for index,row in data_to_process.iterrows():
            tf_dict = dict(row)
            tfidf = {}
            for word, val in tf_dict.items():
                if word  != 'label'  :
                    tfidf[word] = val*idfDict[word]*(cf_data_to_process[word]/no_records_per_label)
                    # print((cf_data_to_process[word]))
            tfidf_array.append(tfidf)
        # return tfidf

    return (word_count,tf_array,idf_array,tfidf_array,word_count_label)

    # return [row_list,wordDict]


In [22]:
result = get_tftdfcf(trainDF.iloc[0:1000])
word_count = pd.DataFrame(result[0])
tf_count = pd.DataFrame(result[1])
idf_data = pd.DataFrame(result[2])
tfidf_data = pd.DataFrame(result[3])
cf_data = pd.DataFrame(result[4])

15396


tapes           0.000051
Depends         0.000136
thereit         0.000071
several         0.004024
"Cool"          0.000032
                  ...   
European        0.000096
woman's         0.000039
twin            0.000102
support.        0.000073
pathetic--in    0.000015
Length: 15395, dtype: float64

In [24]:
cf_data

,tapes,Depends,thereit,several,"""Cool""",untamed,stomach,tricky,student,"head.Again,",...,great!:,"rabbit,",presentation,EUREKA!!,A/C,European,woman's,twin,support.,pathetic--in
0,1,0,1,8,1,1,2,1,5,0,...,2,0,0,1,1,2,1,0,2,0
1,0,2,0,11,0,0,2,0,4,1,...,0,1,1,0,0,0,0,2,0,1


In [25]:
idf_data

,tapes,Depends,thereit,several,"""Cool""",untamed,stomach,tricky,student,"head.Again,",...,great!:,"rabbit,",presentation,EUREKA!!,A/C,European,woman's,twin,support.,pathetic--in
0,3.0,3.0,3.0,1.721246,3.0,3.0,2.39794,3.0,2.045757,3.0,...,2.69897,3.0,3.0,3.0,3.0,2.69897,3.0,2.69897,2.69897,3.0


In [26]:
word_count

,tapes,Depends,thereit,several,"""Cool""",untamed,stomach,tricky,student,"head.Again,",...,great!:,"rabbit,",presentation,EUREKA!!,A/C,European,woman's,twin,support.,pathetic--in
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
tf_count

,tapes,label,Depends,thereit,several,"""Cool""",untamed,stomach,tricky,student,...,great!:,"rabbit,",presentation,EUREKA!!,A/C,European,woman's,twin,support.,pathetic--in
0,0.0,__label__2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,__label__2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,__label__2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,__label__2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,__label__2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,__label__1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
996,0.0,__label__1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,0.0,__label__2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
998,0.0,__label__1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
